In [12]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape, x_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28)


In [14]:
import numpy as np
x_train = x_train[:,:,:,np.newaxis]/255.
x_test = x_test[:,:,:,np.newaxis]/255.
print(x_train.shape, y_train.shape, x_test.shape)

(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1)


In [4]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

cp = ModelCheckpoint("./mnist_best.hdf5", monitor='loss', save_best_only=True)
esc = EarlyStopping(monitor='loss', patience=10)
history = model.fit(x_train,y_train,
                    validation_data=(x_test,y_test),
                    batch_size = 64,
                    epochs=1000, callbacks=[esc, cp])

Epoch 1/1000
938/938 [==============================] - 11s 12ms/step - loss: 0.1787 - accuracy: 0.9473 - val_loss: 0.0729 - val_accuracy: 0.9781
Epoch 2/1000
938/938 [==============================] - 11s 12ms/step - loss: 0.0552 - accuracy: 0.9838 - val_loss: 0.0637 - val_accuracy: 0.9793
Epoch 3/1000
938/938 [==============================] - 12s 12ms/step - loss: 0.0340 - accuracy: 0.9892 - val_loss: 0.0632 - val_accuracy: 0.9802
Epoch 4/1000
938/938 [==============================] - 12s 12ms/step - loss: 0.0220 - accuracy: 0.9931 - val_loss: 0.0608 - val_accuracy: 0.9838
Epoch 5/1000
938/938 [==============================] - 11s 12ms/step - loss: 0.0146 - accuracy: 0.9955 - val_loss: 0.0669 - val_accuracy: 0.9797
Epoch 6/1000
938/938 [==============================] - 11s 12ms/step - loss: 0.0112 - accuracy: 0.9967 - val_loss: 0.0642 - val_accuracy: 0.9833
Epoch 7/1000
938/938 [==============================] - 11s 12ms/step - loss: 0.0075 - accuracy: 0.9977 - val_loss: 0.0725 -

KeyboardInterrupt: 

In [11]:
from tensorflow.keras.models import load_model
mnist_model = load_model("./mnist_best.hdf5")

In [15]:
from sklearn.metrics import f1_score
from sklearn import metrics
pred = mnist_model.predict(x_test)

pred_idx = np.argmax(pred, axis=1)

f1 = f1_score(y_test, pred_idx, average='macro')
print('f1 score :', f1)
print(metrics.classification_report(y_test, pred_idx))

f1 score : 0.9835392374920339
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.99      0.98      1010
           4       0.99      0.98      0.99       982
           5       0.97      0.99      0.98       892
           6       0.99      0.98      0.99       958
           7       0.97      0.99      0.98      1028
           8       1.00      0.96      0.98       974
           9       0.98      0.98      0.98      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [17]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(mnist_model)
mnist_tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\etriai03\AppData\Local\Temp\tmpseekj1vz\assets


In [19]:
with open('./mnist_tflite_model.tflite', 'wb') as f:
    f.write(mnist_tflite_model)

In [20]:
interpreter = tf.lite.Interpreter(model_path='./mnist_tflite_model.tflite')
interpreter.allocate_tensors()

In [21]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_details

[{'name': 'conv2d_1_input',
  'index': 0,
  'shape': array([ 1, 28, 28,  1]),
  'shape_signature': array([-1, 28, 28,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [22]:
output_details

[{'name': 'Identity',
  'index': 12,
  'shape': array([ 1, 10]),
  'shape_signature': array([-1, 10]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [29]:
input_shape=(1,28,28,1)

In [30]:
input_data = np.array(x_test[0].reshape(input_shape),dtype=input_details[0]['dtype'])

In [31]:
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

In [33]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[7.6481340e-23 1.0595977e-20 3.2470803e-17 6.9597164e-17 1.1869738e-20
  1.0972019e-20 6.8836209e-32 1.0000000e+00 2.2814787e-18 1.6040564e-12]]


In [35]:
pred = mnist_model.predict(x_test[:1])
pred

array([[7.6480759e-23, 1.0595856e-20, 3.2470555e-17, 6.9596369e-17,
        1.1869557e-20, 1.0971936e-20, 6.8835680e-32, 1.0000000e+00,
        2.2814524e-18, 1.6040443e-12]], dtype=float32)